# Xpand IT DS Challenge

This notebook contains the template you should use to present your code, results and conclusions. You should keep the main structure intact to make it easier to evaluate and compare in the end.

## Dataset
You can find the dataset in the data folder of the repository. The folder contains two files:
* dow_jones_index.data - dataset data
* dow_jones_index.names - dataset information and details

## Business Analysis
Here you should conduct a brief analysis of what is Dow Jones Index. You can enumerate the main topics to take into account based on the dataset provided as well as your understandings of the variables.


-----
*Add here your business analysis conclusions (max. 200 words)*

-----


## Data Understanding
During the data understanding phase, you should focus on understanding what each variable represents, compute statistics and visualizations. Some questions that may guide your work follow:
* Feature engineering: should new features be created from the existing ones?
* What will be your features and your label?
* Is the dataset ready for the prediction task? (ex: missing values)
* How will the data be split into train and test sets?

-----

In [ ]:
#Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.figure as fgr
import seaborn as sns

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, mean_squared_error, root_mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

### Data Loading

In [ ]:
df = pd.read_csv("data/dow_jones_index.data", sep=',')
print(df.shape)
print(df.head(5))

### Data Inspection

In [1]:
#Check data types and missing values
df.info()

NameError: name 'df' is not defined

In [ ]:
#Return number of missing values for each column
df.isna().sum()

### Data Cleaning

In [ ]:
#Convert stock to category and date to datetime
df['stock'] = df['stock'].astype('category')
df['date'] = pd.to_datetime(df['date'])

#Select all the remeining object columns (which are actually currency columns)
currency_features = df.select_dtypes(include=['object']).columns
#Remove the dollar sign from currency columns and convert to float
df[currency_features] = df[currency_features].replace('[\$,]', '', regex=True).astype(float)

#Check the types
print(df.dtypes)

### Exploratory Data Analysis (EDA)

In [ ]:
#Summary statistics for the dataset
df.describe().T.apply(lambda s: s.apply('{0:.3f}'.format))

In [ ]:
#Function to plot the correlation matrix heatmap of all numerical variables in data
def correlation_matrix(df):
    columns_of_interest = df.select_dtypes(include=[np.number]).columns
    df_selected = df[columns_of_interest]

    # Calculate the correlation matrix
    correlation_matrix = df_selected.corr(method = 'pearson')

    # Configure the size of the figure
    plt.figure(figsize=(14, 10))

    # Correlation heatmap
    sns.heatmap(correlation_matrix, annot=True, cmap='viridis', fmt='.2f')
    plt.title('Correlation Matrix Heatmap')
    plt.show()

correlation_matrix(df)

In [ ]:
#Function to create violin plots of all numerical variables in data, checking distribution and density
def plot_violins(data):
    columns_of_interest = data.select_dtypes(include=[np.number]).columns
    plt.figure(figsize=(18, 18))

    # Loop to create boxplots for each numeric column
    for i, column in enumerate(columns_of_interest, 1):
        plt.subplot(4, 4, i)
        sns.violinplot(data=data, y=column, palette='viridis')
        plt.title(f'Boxplot of {column}')
        plt.ylabel(column)

    plt.tight_layout()
    plt.grid(False)
    plt.show()
    
plot_violins(df)

#Line plot of stock closing prices over time
def line_plot_feature(data, column):
    sns.set_theme(rc={'figure.figsize':(15, 10)})
    lineplot = sns.lineplot(data=data, x='date', y= column, hue='stock')
    plt.title('Stock Closing Prices Over Time')
    plt.show()

line_plot_feature(df, 'close')

In [ ]:
line_plot_feature(df, 'percent_change_price')

In [ ]:
columns_of_interest = df.select_dtypes(include=[np.number]).columns
sns.pairplot(df[columns_of_interest])
plt.show()

In [ ]:
# Define the viridis colormap
viridis_palette = sns.color_palette("viridis", as_cmap=True)

# Set up the matplotlib figure
plt.figure(figsize=(15, 12))

# Create a pair plot with the viridis colormap
pair_plot = sns.pairplot(df[['percent_change_next_weeks_price', 'percent_change_price', 'previous_weeks_volume']], 
                         diag_kind='kde', 
                         markers='o',
                         palette=viridis_palette)


### Feature Engineering

In [ ]:
#Calculate volatility (high-low spread)
df['volatility'] = df['high'] - df['low']

#Rolling averages and volatility over 3 weeks
df['rolling_avg_3w'] = df.groupby('stock')['close'].rolling(window=3).mean().reset_index(0, drop=True)
df['rolling_vol_3w'] = df.groupby('stock')['volatility'].rolling(window=3).mean().reset_index(0, drop=True)

#df['ma_7'] = df.groupby('stock')['close'].rolling(window=7).mean().reset_index(0, drop=True)
#df['ma_3'] = df.groupby('stock')['close'].rolling(window=3).mean().reset_index(0, drop=True)

#Lag features of previous week's percent change
df['lag_1'] = df.groupby('stock')['percent_change_price'].shift(1)
df['lag_2'] = df.groupby('stock')['percent_change_price'].shift(2)

In [ ]:
correlation_matrix(df)

-----
*Add here your data understanding findings and conclusions (max. 200 words)*

-----

## Modelling
In this phase, your main goal is to develop and describe your approach to the solution of the problem. Some guidelines to help you:
* What metrics will you use to evaluate your solutions?
* What are some algorithms that can lead to good results? And why?
* Describe in detail your thought process during the development of your solution.
* Present your results.


-----


### XGBoost

In [ ]:
#Create target column: 1 if it's the best-performing stock for that week, otherwise 0
df['best_stock'] = df.groupby('date')['percent_change_next_weeks_price'].transform(lambda x: (x == x.max()).astype(int))

In [ ]:
#Define features and target
features_with_date = ['date', 'stock', 'open', 'high', 'low', 'close', 'volume', 'percent_change_price', 
            'percent_change_volume_over_last_wk', 'previous_weeks_volume', 
            'days_to_next_dividend', 'percent_return_next_dividend', 'volatility', 'rolling_avg_3w', 'rolling_vol_3w', 'lag_1', 'lag_2']


features = ['open', 'high', 'low', 'close', 'volume', 'percent_change_price', 
            'percent_change_volume_over_last_wk', 'previous_weeks_volume', 
            'days_to_next_dividend', 'percent_return_next_dividend', 'volatility', 'rolling_avg_3w', 'rolling_vol_3w', 'lag_1', 'lag_2']

target = 'best_stock'

#Create a copy of dataframe and drop missing values
df_xgb = df.copy()
df_xgb = df_xgb.dropna()

X = df_xgb[features_with_date]
y = df_xgb[target]

# Train-test split - first 70% for training, last 20% for testing
train_size = int(len(df_xgb) * 0.7)
train_data = df_xgb.iloc[:train_size]
test_data = df_xgb.iloc[train_size:]

X_train = train_data[features_with_date]
y_train = train_data[target]
X_test = test_data[features_with_date]
y_test = test_data[target]

#smote = SMOTE(random_state=2)
#X_train_smote, y_train_smote = smote.fit_resample(X_train[features], y_train)

In [ ]:
#Train XGBoost model
model = XGBClassifier(scale_pos_weight = 1.5, use_label_encoder=False)
model.fit(X_train[features], y_train)

#Evaluate performance
y_pred = model.predict(X_test[features])
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))

In [ ]:
y_pred

In [ ]:
#Accuracy, precision, recall
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1:.2f}')

data_test = X_test.copy()  # Copy test data
data_test['predicted_best_stock'] = y_pred

#Merge with the original data to get the actual returns for each predicted stock
data_test = data_test.merge(df[['date', 'stock', 'percent_change_next_weeks_price']], on=['date','stock'], how='left')

#Assuming 100€ investment each week
data_test['predicted_return'] = data_test['percent_change_next_weeks_price']
cumulative_return = data_test[data_test['predicted_best_stock']==1]['predicted_return'].sum()
print(f'Cumulative Return: {cumulative_return} EUR')

### Time Series Forecasting with LSTM

In [ ]:
#Create a copy of dataframe then sort and drop missing values
df_lstm = df.copy()
df_lstm.set_index('date', inplace=True)
df_lstm.sort_index(axis=0,inplace=True)

features = ['close','volume','volatility','rolling_vol_3w','rolling_avg_3w','percent_change_price','previous_weeks_volume','percent_return_next_dividend','percent_change_next_weeks_price']
target = 'percent_change_next_weeks_price'

df_lstm = df_lstm[features]
df_lstm.fillna(0, inplace=True)

#Preprocessing for LSTM
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_lstm)

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length][:-1]) # Use seq_length data points as input (exclude the target)
        y.append(data[i+seq_length][-1]) #Target is 'percent_change_next_weeks_price' (the last column)
    return np.array(X), np.array(y)

seq_length = 4  #use the last 4 weeks to predict the next week
X, y = create_sequences(scaled_data, seq_length)

# Split into training and testing sets
train_size = int(0.7 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

#Define LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=120, batch_size=1, verbose=1)

# Make predictions
y_pred = model.predict(X_test)

In [ ]:
#Invert the scaling of the predicted data (back to original scale)
y_pred_rescaled = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_pred), axis=1))[:, -1]
y_test_rescaled = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1))[:, -1]

#Plot the predicted vs actual values
plt.figure(figsize=(12, 6))
plt.plot(y_test_rescaled, color='blue', label='Actual percent_change_next_weeks_price')
plt.plot(y_pred_rescaled, color='red', label='Predicted percent_change_next_weeks_price')
plt.title('LSTM Model - Predicted vs Actual Percent Change Next Week\'s Price')
plt.xlabel('Time')
plt.ylabel('Percent Change Next Week\'s Price')
plt.legend()
plt.show()

rmse = mean_squared_error(y_test_rescaled, y_pred_rescaled, squared=False)
print(f'Root Mean Squared Error (MSE): {rmse}')

In [ ]:
def cumulative_returns(data):

    data['best_actual_stock'] = data.groupby('date')['percent_change_next_weeks_price'].transform(lambda x: (x == x.max()).astype(int))
    data['best_predicted_stock'] = data.groupby('date')['predicted_change_next_weeks_price'].transform(lambda x: (x == x.max()).astype(int))

    actual_returns_add = data[data['best_actual_stock']==1]['percent_change_next_weeks_price'].sum()
    predicted_returns_add = data[data['best_predicted_stock']==1]['percent_change_next_weeks_price'].sum()

    actual_returns_array = data[data['best_actual_stock']==1]['percent_change_next_weeks_price'].to_numpy()
    predicted_returns_array = data[data['best_predicted_stock']==1]['percent_change_next_weeks_price'].to_numpy()

    # Cumulative returns for both actual and predicted values (assuming initial investment of 100€)
    initial_investment = 100
    cumulative_actual_return = np.cumsum(actual_returns_array) * initial_investment / 100 + initial_investment
    cumulative_predicted_return = np.cumsum(predicted_returns_array) * initial_investment / 100 + initial_investment

    # Plot cumulative returns
    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_actual_return, label='Actual Cumulative Return', color='blue')
    plt.plot(cumulative_predicted_return, label='Predicted Cumulative Return', color='red')
    plt.title('Cumulative Returns (Actual vs Predicted)')
    plt.xlabel('Weeks')
    plt.ylabel('Cumulative Return (€)')
    plt.legend()
    plt.show()

    # Step 7: Evaluate model with Mean Squared Error (MSE) as an additional metric
    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(actual_returns_array, predicted_returns_array)
    print(f'Mean Squared Error (MSE): {mse}')
    rmse = root_mean_squared_error(actual_returns_array, predicted_returns_array)
    print(f'Root Mean Squared Error (RMSE): {rmse}')

    # Final cumulative returns
    final_actual_return = cumulative_actual_return[-1]
    final_predicted_return = cumulative_predicted_return[-1]

    print(f"Final Actual Cumulative Return: {final_actual_return:.2f} EUR")
    print(f"Final Predicted Cumulative Return: {final_predicted_return:.2f} EUR")
    print(f"Percentage of MAX returns acheived: {(100*predicted_returns_add)/actual_returns_add:.2f} %")

    print(cumulative_actual_return)
    print(cumulative_predicted_return)

In [ ]:
df_lstm_test = df_lstm[train_size:]
df_lstm_test = df_lstm_test[seq_length:]

df_lstm_test['predicted_change_next_weeks_price'] = y_pred_rescaled

cumulative_returns(df_lstm_test)

### Time Series Forecasting with Prophet

In [ ]:
from prophet import Prophet

df_prophet = df.copy()
df_prophet= df_prophet.dropna()
df_prophet.sort_values(by=['date'], inplace=True)

#Remove extreme outliers based on the percent change
#remove changes greater than 3 standard deviations from the mean
mean_change = df_prophet['percent_change_next_weeks_price'].mean()
std_change = df_prophet['percent_change_next_weeks_price'].std()

df_prophet = df_prophet[(df_prophet['percent_change_next_weeks_price'] > mean_change - 3 * std_change) &
            (df_prophet['percent_change_next_weeks_price'] < mean_change + 3 * std_change)]

#Split the dataset into training and testing based on date
split = pd.to_datetime('2011-05-21')
train_data = df_prophet[df_prophet.date < split]
test_data = df_prophet[df_prophet.date >= split]


predictions = pd.DataFrame()

#Loop over each stock to fit a Prophet model and make predictions
for stock in df_prophet['stock'].unique():
    #print(f"Running Prophet model for stock: {stock}")
    
    #Filter data for the current stock
    stock_train = train_data[train_data['stock'] == stock]
    stock_test = test_data[test_data['stock'] == stock]
    
    
    #Prepare data for Prophet
    prophet_data = stock_train[['date', 'percent_change_next_weeks_price','close', 'volatility', 'rolling_avg_3w','lag_1']].copy()
    #Prophet requires 'ds' for date and 'y' for target
    prophet_data.columns = ['ds', 'y', 'close', 'volatility', 'rolling_avg_3w','lag_1']
    
    #Initialize and fit Prophet model with aditional regressors
    model = Prophet(weekly_seasonality=True)
    model.add_regressor('close')
    model.add_regressor('volatility')
    model.add_regressor('rolling_avg_3w')
    model.add_regressor('lag_1')
    model.fit(prophet_data)
    
    # Create future dataframe to predict for test set
    future = pd.DataFrame(stock_test[['date', 'close', 'volatility', 'rolling_avg_3w','lag_1']])
    future.columns = ['ds', 'close', 'volatility', 'rolling_avg_3w','lag_1']
    #Make predictions
    forecast = model.predict(future)
   
    #Store predictions with the stock information
    stock_test['predicted_change_next_weeks_price'] = forecast['yhat_lower'].values
    predictions = pd.concat([predictions, stock_test], axis=0)

In [ ]:
#Plot actual vs predicted values for each stock in the test set
for stock in df_prophet['stock'].unique():
    stock_predictions = predictions[predictions['stock'] == stock]
    
    plt.figure(figsize=(12, 6))
    plt.plot(stock_predictions['date'], stock_predictions['percent_change_next_weeks_price'], label='Actual Percent Change', color='blue')
    plt.plot(stock_predictions['date'], stock_predictions['predicted_change_next_weeks_price'], label='Predicted Percent Change', color='red')
    plt.title(f'Stock: {stock} - Actual vs Predicted Percent Change')
    plt.xlabel('Date')
    plt.ylabel('Percent Change Next Week\'s Price')
    plt.legend()
    plt.show()

In [ ]:
mse = mean_squared_error(stock_predictions['percent_change_next_weeks_price'], stock_predictions['predicted_change_next_weeks_price'])
print(f"Mean Squared Error for stocks: {mse}")

rmse = root_mean_squared_error(stock_predictions['percent_change_next_weeks_price'], stock_predictions['predicted_change_next_weeks_price'])
print(f'Root Mean Squared Error (RMSE): {rmse}')

In [ ]:
predictions.sort_values(by=['date'], inplace=True)
cumulative_returns(predictions)

-----
*Add here your modelling results and conclusions (max. 200 words)*

-----

## Conclusions
In the conclusions, you should enumerate the results you got after completing the challenge.
* How good do you consider your results? 
* What are some factors that would contribute to get better results?
* What are some advantages and disadvantages of your solution?
* What can be done as future work to improve your results?


-----
*Add here your final conclusions (max. 400 words)*

-----

#### Feedback

-----
*Add here your thoughts and feedback regarding this challenge.*

-----

To submit your solution you should e-mail us this notebook in response to the e-mail you initially received with the challenge.